# AstrbotTasks
- Taskiq再封装测试

首先要特别注意：
    InmemoryBroker比broker标准多几个方法，并且不可监听！


In [26]:
from taskiq import InMemoryBroker

# 基本任务分发代理
broker = InMemoryBroker()


# InmemoryResultBackend
- 作用：用于 InMemoryBroker 的结果存储，所有结果都保存在内存的有序字典中。
- 最大存储数：默认最多保存 100 条结果（可通过 max_stored_results 参数调整，-1 表示无限制）。
## 主要属性：
- results：保存任务结果的 OrderedDict，key 为 task_id。
- progress：保存任务进度的 OrderedDict，key 为 task_id。
## 核心方法：
- set_result(task_id, result)：保存任务结果，超出最大数量时自动移除最早的结果。
- is_result_ready(task_id)：检查指定任务结果是否已就绪。
- get_result(task_id)：获取指定任务的结果，不存在则抛异常。
- set_progress(task_id, progress)：保存任务进度，超出最大数量时自动移除最早的进度。
- get_progress(task_id)：获取指定任务的进度，若无则返回 None。
# InMemoryBroker
## 作用：本地开发用的内存型任务分发代理，不依赖外部消息队列。
## 主要参数：
- sync_tasks_pool_size：同步任务线程池大小，默认 4。
- max_stored_results：最大结果存储数，默认 100。
- max_async_tasks：最大异步任务数，默认 30。
- await_inplace：是否在 kick 时直接 await 任务。
## 核心属性：
- result_backend：结果后端，使用 InmemoryResultBackend。
- executor：线程池执行器。
- receiver：任务接收器，负责参数校验和任务调度。
- _running_tasks：当前正在运行的异步任务集合。
## 核心方法：
- kick(message)：执行指定任务，支持 await_inplace 或异步调度。
- listen()：不支持监听，调用会抛出 RuntimeError。<span style="color:red">(特别注意！其他broker都行！)</span>
- wait_all()：等待所有正在运行的任务完成，常用于测试场景。<span style="color:red">（特！别！注！意！其他！broker!没!）</span>
- startup() / shutdown()：分别触发启动和关闭事件，管理生命周期。

In [27]:

# 定义任务

@broker.task("sample_task", a="label_a", b="label_b")
async def sample_task(x: int, y: int) -> int:
    return x + y


# 下面假设在另一个文件发送（不！可！以！直接导入任务！）
- 可以通过事件调度（内置事件）
- 可以通过任务名调度

In [ ]:

from typing import Any
from taskiq import AsyncTaskiqDecoratedTask, AsyncTaskiqTask, TaskiqEvents, TaskiqResult, TaskiqState


@broker.on_event(TaskiqEvents.CLIENT_STARTUP)
async def on_startup(state: TaskiqState):
    print("Broker is starting up...")

@broker.on_event(TaskiqEvents.CLIENT_STARTUP)
async def on_client_startup(state: TaskiqState):
    print("Client is starting up 2...")

@broker.on_event(TaskiqEvents.WORKER_SHUTDOWN)
async def on_shutdown(state: TaskiqState):
    print("Broker is shutting down...")


await broker.startup()

await broker.shutdown()

st: AsyncTaskiqDecoratedTask[Any, Any] | None = broker.find_task("sample_task")
if st is None:
    raise ValueError("Task not found")
await st.kiq(x=1, y=2)
task: AsyncTaskiqTask[Any] = await st.kiq(x=1, y=2)
result: TaskiqResult[Any] = await task.wait_result()

print(f"Task result: {result}")  # 输出: Task result: 
print(f"Task result: {result.return_value}")  # 输出: 3



Broker is starting up...
Client is starting up 2...
Broker is shutting down...
Task result: is_err=False log=None return_value=3 execution_time=0.0 labels={'a': 'label_a', 'b': 'label_b'} error=None
Task result: 3
